In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import ShuffleSplit

In [2]:
employee_data = pd.read_csv("../Data/regression_employee_data.csv")

In [3]:
employee_data.head()

,Emp #,Designation,Experience,Salary,AggrBehavScore,ActivityScore,LinesOfCode,Qualification,Degree,YearsOfStudy,EducationalInstitute,Gender,Empl_Band
0,e1,Director,16.0,85861,0.526928,5,1678,Grad,NaN,15.0,clg,M,A
1,e2,Jr.Dev,3.0,25376,6.420421,18,6082,Grad,Btech,16.0,clg,F,C
2,e3,Sr.Dev,6.0,38193,2.802337,7,2927,PostGrad,Msc,17.0,unv,M,B
3,e4,Jr.Dev,3.0,22807,9.374888,23,10249,NaN,Mtech,18.0,NaN,F,C
4,e5,Sr.Dev,4.0,37624,3.336564,10,3159,NaN,Btech,16.0,NaN,F,B


* Step 1: Split data into Train and Test

In [4]:
shuffleSplit = ShuffleSplit(n_splits=1, test_size=0.2, random_state=33)
for train_index, test_index in shuffleSplit.split(employee_data):
    train_set = employee_data.loc[train_index]
    test_set = employee_data.loc[test_index]

In [5]:
train_set.shape

(1209, 13)

In [6]:
test_set.shape

(303, 13)

* Impute null

In [7]:
train_set.loc[train_set[train_set['LinesOfCode'] < 0].index.values, 'LinesOfCode'] = 0

In [8]:
YearsOfStudy_impute_value = train_set['YearsOfStudy'].mode().values[0]
train_set['YearsOfStudy'].fillna(YearsOfStudy_impute_value, inplace=True)

In [9]:
Qualification_impute_value = train_set['Qualification'].mode().values[0]
train_set['Qualification'].fillna(Qualification_impute_value, inplace=True)

In [10]:
Degree_impute_value = train_set['Degree'].mode().values[0]
train_set['Degree'].fillna(Degree_impute_value, inplace=True)

In [11]:
EducationalInstitute_impute_value = train_set['EducationalInstitute'].mode().values[0]
train_set['EducationalInstitute'].fillna(EducationalInstitute_impute_value, inplace=True)

* Convert all categorical features to numbers

In [12]:
train_set.drop('Emp #', axis=1, inplace=True)

* Let's convert categorical columns to numbers (Label Encoding)

In [13]:
cat_list_dict = {'Designation': ['Sr.Dev', 'Analyst', 'Sr.Analyst', 'Manager', 'Jr.Dev', 'Director', 'Sr.Manager'],
                'Qualification': ['Grad', 'PostGrad', 'Phd', 'postdoc'],
                'Degree': ['Btech', 'Bsc', 'Msc', 'Mba', 'IntMsc', 'B.A', 'Mtech'],
                'EducationalInstitute' : ['unv', 'iit', 'clg', 'cunv'],
                'Gender': ['M', 'F'],
                'Empl_Band': ['B', 'C', 'A']}

In [14]:
def convertCategorisToCodes(columns, df):
    for column in columns:
        df[column] = pd.Categorical(df[column], categories = cat_list_dict[column])
        df[column] = df[column].cat.codes

In [15]:
convertCategorisToCodes(train_set.select_dtypes(['object']).columns.values, train_set)

* Let's build a Polynomial Regressin model

In [16]:
train_X = train_set.loc[:, ['Designation', 'Experience', 'AggrBehavScore', 'ActivityScore', 'LinesOfCode', 'Qualification',
 'Degree', 'YearsOfStudy', 'EducationalInstitute', 'Gender', 'Empl_Band']]
train_y = train_set['Salary']

* When we raise the dataset to "d" - degree polynomial, the number of columns in dataset becomes to (n + d)!/n! d!

In [17]:
train_X.shape

(1209, 11)

In [18]:
from sklearn.preprocessing import PolynomialFeatures
poly_features = PolynomialFeatures(degree=2, include_bias=True)
train_X_poly = poly_features.fit_transform(train_X)
train_X_poly.shape

(1209, 78)

In [19]:
poly_model = LinearRegression()
poly_model.fit(train_X_poly, train_y)

LinearRegression()

In [20]:
train_prediction = poly_model.predict(train_X_poly)

In [21]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

In [22]:
train_r2_score = r2_score(train_y, train_prediction)
train_mse = mean_squared_error(train_y, train_prediction)

print("Train R2 Score : ", train_r2_score)
print("Train Mean Squared Error : ", train_mse)
print("Train Root Mean Squared Error : ", np.sqrt(train_mse))

Train R2 Score :  0.9089509097675514
Train Mean Squared Error :  33975622.82261561
Train Root Mean Squared Error :  5828.861194317086


* Let's test the model on TEST set

In [23]:
test_set.loc[test_set[test_set['LinesOfCode'] < 0].index.values, 'LinesOfCode'] = 0
test_set.drop('Emp #', axis=1, inplace=True)
test_set['YearsOfStudy'].fillna(YearsOfStudy_impute_value, inplace=True)
test_set['Qualification'].fillna(Qualification_impute_value, inplace=True)
test_set['Degree'].fillna(Degree_impute_value, inplace=True)
test_set['EducationalInstitute'].fillna(EducationalInstitute_impute_value, inplace=True)
convertCategorisToCodes(test_set.select_dtypes(['object']).columns.values, test_set)

In [24]:
test_X = test_set.loc[:,['Designation', 'Experience', 'AggrBehavScore', 'ActivityScore', 'LinesOfCode', 'Qualification',
 'Degree', 'YearsOfStudy', 'EducationalInstitute', 'Gender', 'Empl_Band']]
test_y = test_set['Salary']

In [25]:
test_X.dtypes

Designation                int8
Experience              float64
AggrBehavScore          float64
ActivityScore             int64
LinesOfCode               int64
Qualification              int8
Degree                     int8
YearsOfStudy            float64
EducationalInstitute       int8
Gender                     int8
Empl_Band                  int8
dtype: object

In [26]:
test_X_poly = poly_features.transform(test_X)
test_X_poly.shape

(303, 78)

In [27]:
test_prediction = poly_model.predict(test_X_poly)

In [28]:
test_r2_score = r2_score(test_y, test_prediction)
test_mse = mean_squared_error(test_y, test_prediction)

print("TEST R2 Score : ", test_r2_score)
print("TESt Mean Squared Error : ", test_mse)
print("TEST Root Mean Squared Error : ", np.sqrt(test_mse))

TEST R2 Score :  0.9103571772225845
TESt Mean Squared Error :  35230128.041615605
TEST Root Mean Squared Error :  5935.497286800459


In [29]:
import pickle

In [30]:
poly_features_out = open("poly_feature_transfomar.pkl","wb")
poly_model_out = open("poly_reg_model.pkl","wb")

pickle.dump(poly_features, poly_features_out)
pickle.dump(poly_model, poly_model_out)

poly_features_out.close()
poly_model_out.close()

### Simmulated Production Move

In [31]:
prod_employee_data = pd.read_csv("../Data/regression_employee_data.csv")

In [32]:
prod_employee_data.loc[test_set[test_set['LinesOfCode'] < 0].index.values, 'LinesOfCode'] = 0
prod_employee_data.drop('Emp #', axis=1, inplace=True)
prod_employee_data['YearsOfStudy'].fillna(YearsOfStudy_impute_value, inplace=True)
prod_employee_data['Qualification'].fillna(Qualification_impute_value, inplace=True)
prod_employee_data['Degree'].fillna(Degree_impute_value, inplace=True)
prod_employee_data['EducationalInstitute'].fillna(EducationalInstitute_impute_value, inplace=True)
convertCategorisToCodes(prod_employee_data.select_dtypes(['object']).columns.values, test_set)

In [33]:
prod_employee_data_X = test_set.loc[:,['Designation', 'Experience', 'AggrBehavScore', 'ActivityScore', 'LinesOfCode', 'Qualification',
 'Degree', 'YearsOfStudy', 'EducationalInstitute', 'Gender', 'Empl_Band']]
#prod_employee_data_y = test_set['Salary']

In [34]:
poly_features_in = open("poly_feature_transfomar.pkl","rb")
poly_model_in = open("poly_reg_model.pkl","rb")

poly_features_pkl = pickle.load(poly_features_in)
poly_model_pkl = pickle.load(poly_model_in)

#prod_X = ...
prod_X_poly = poly_features_pkl.transform(prod_employee_data_X)
prod_prediction = poly_model_pkl.predict(prod_X_poly)

In [35]:
prod_prediction

array([107075.05615359, 103541.22005823, 158199.01212844, 104535.97789399,
        98309.4601377 , 183898.21130624,  99593.92196134, 109829.78169681,
       101653.38967258,  92690.70868225, 154841.24488418, 115907.21181935,
       123971.38979537, 168466.75791128, 104392.52523513, 130755.47660957,
       197854.64162302, 105835.93062449, 134466.18617575, 103453.31407893,
       106261.34795128, 333123.11445163, 118379.61046964, 101235.7101164 ,
       101954.24285083, 344759.11168813, 133652.90240561,  80545.4375483 ,
        94285.00920153, 139156.01247513, 180631.00718334, 153400.28016804,
        91467.90976986, 110896.08906912, 131322.440331  , 117675.89559502,
        80884.04719366, 128039.64943549, 127137.8968465 , 113247.34226384,
        80897.11193447, 151481.97219697, 107646.80248815, 165600.18772153,
       101253.03591973, 100893.45205284,  98668.23728198,  90769.33513725,
       150599.79440122,  94115.1254356 ,  93231.34177011, 119043.30560146,
       286652.03440974,  

https://services.math.duke.edu/education/ccp/materials/mvcalc/surfaces/surf2.html